# Problem description:
### Suppose that Alice wants to send two messages to Bob. Also, She wants for Bob to be able to read only one of her messages depending on Bob's choice. We also want to avoid Bob reading the two original messages from Alice, and avoid Alice being aware of the choice of Bob. Here we have the assumption that Bob will play by the rules no matter what.

# Example:
### Supose that Alice wants to send one of the two strings "I like dogs" and "I like cats" to Bob. Bob made his choice to read the message 0. So Alice sends them both encrypted (to avoid Alice knowing the choice of Bob) and Bob will only be able to read the message 0 but not both.

# Protocol for the example

## Step zero: Imports and the declaration of two functions to convert from strings to bits and from bits to strings

In [44]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from random import randint
import hashlib
#These two functions were taken from https://stackoverflow.com/questions/10237926/convert-string-to-list-of-bits-and-viceversa
def tobits(s):
    result = []
    for c in s:
        bits = bin(ord(c))[2:]
        bits = '00000000'[len(bits):] + bits
        result.extend([int(b) for b in bits])
    return ''.join([str(x) for x in result])

def frombits(bits):
    chars = []
    for b in range(int(len(bits) / 8)):
        byte = bits[b*8:(b+1)*8]
        chars.append(chr(int(''.join([str(bit) for bit in byte]), 2)))
    return ''.join(chars)

## First step: Alice chooses two messages. Then She sends to Bob n qubits in 2 random basis.

In [56]:
#Alice cell, Bob can't see what's going in on here
m0 = tobits("I like dogs") 
m1 = tobits("I like cats")
messageMerge = m0+m1
Alice_bases = [randint(0,1) for x in range(len(messageMerge))]
qubits = list()
for i in range(len(Alice_bases)):
    mycircuit = QuantumCircuit(1,1)
    if(Alice_bases[i] == 0):
        if(messageMerge[i] == "1"):
            mycircuit.x(0)
    else:
        if(messageMerge[i] == "0"):
            mycircuit.h(0)
        else:
            mycircuit.x(0)
            mycircuit.h(0)
    qubits.append(mycircuit)

## Second step: Bob receives Alice qubits and then He meassures them according to a random selection of basis.

In [57]:
#Bob cell, Alice can't see what's going in on here
Bob_bases = [randint(0,1) for x in range(176)]
backend = Aer.get_backend('qasm_simulator')
measurements = list()
choice = 0
for i in range(len(Bob_bases)):
    qubit = qubits[i]
    if(Bob_bases[i] == 0):
        qubit.measure(0,0)
    else:
        qubit.h(0)
        qubit.measure(0,0)
    result = execute(qubit, backend, shots=1, memory=True).result()
    measurements.append(int(result.get_memory()[0]))

## Third step: Alice sends to Bob the bases She used to measure. Then Bob stores the index of the basis that correspond to the equal values between the ones sent by Alice and the random basis generated by Bob.

In [58]:
#Bob cell, Alice can't see what's going in on here
I0 = list()
I1 = list()
if(choice == 0):    
    for i in range(len(Alice_bases)):
        if(Alice_bases[i] == Bob_bases[i]):
            I0.append(i)
        else:
            I1.append(i)
else:
    for i in range(len(Alice_bases)):
        if(Alice_bases[i] == Bob_bases[i]):
            I1.append(i)
        else:
            I0.append(i)

## Four step: Alice receives I0 and I1 (the set of the index of equal basis and the set of the index of distinct basis). Then she prepares her final message to Bob.

In [59]:
#Alice cell, Bob can't see what's going in on here
x0 = list()
for x in I0:
    x0.append(messageMerge[x])
x1 = list()
for x in I1:
    x1.append(messageMerge[x])
fx0 = ''.join(format(ord(i), 'b') for i in hashlib.sha224(''.join(x0).encode('utf-8')).hexdigest())
fx1 = ''.join(format(ord(i), 'b') for i in hashlib.sha224(''.join(x1).encode('utf-8')).hexdigest())
s0 = ''
s1 = ''
for bit in range(len(m0)):
    s0 += str(int(fx0[bit]) ^ int(m0[bit]))
for bit in range(len(m1)):
    s1 += str(int(fx1[bit]) ^ int(m1[bit]))

## Final step: Bob receives Alice messages S0 and S1. Then He applies a XOR in order to get the message He choosed from before.

In [60]:
xB0 = list()
if(choice == 0):
    for x in I0:
        xB0.append(measurements[x])
else:
    for x in I1:
        xB0.append(measurements[x])
fxB0 = ''.join(format(ord(i), 'b') for i in hashlib.sha224(''.join([str(x) for x in xB0]).encode('utf-8')).hexdigest())
mB0 = bytearray()
if(choice == 0):
    for bit in range(len(s0)):
        mB0.append(int(fxB0[bit]) ^ int(s0[bit]))
    print("Alice message was: " + frombits(mB0))
else:
    for bit in range(len(s1)):
        mB0.append(int(fxB0[bit]) ^ int(s1[bit]))
    print("Alice message was: " + frombits(mB0))

Alice message was: I like dogs


## References:
Broadbent, A., Schaffner, C. Quantum cryptography beyond quantum key distribution. Des. Codes Cryptogr. 78, 351–382 (2016). https://doi.org/10.1007/s10623-015-0157-4